In [1]:
import torch
from torch.autograd import Variable
from tqdm.notebook import tqdm
import random
from sklearn.metrics import mean_squared_error

In [2]:
class MatrixFactorization(torch.nn.Module):
    ''' A simple neural network that predicts a rating for a user and item'''
    
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        
        # Create user embeddings: These are the latent factors for users 
        # that capture their preferences in which types of items they prefer.
        self.user_factors = torch.nn.Embedding(n_users, n_factors)

        # Create item embeddings: These are the latent factors for items 
        # that reflect what they are at an implicit level
        self.item_factors = torch.nn.Embedding(n_items, n_factors)

    def forward(self, user, item):
        # Multiply the user and item embeddings to predict the score
        return (self.user_factors(user)*self.item_factors(item)).sum(1)

In [3]:
# Get the data from Canvas or from https://grouplens.org/datasets/movielens/100k/ 
#
# From the README:
#
# MovieLens data sets were collected by the GroupLens Research Project
# at the University of Minnesota.
# 
# This data set consists of:
#        * 100,000 ratings (1-5) from 943 users on 1682 movies. 
#        * Each user has rated at least 20 movies. 
#        * Simple demographic info for the users (age, gender, occupation, zip)

training_user_item_rating_tuples = []
users = set()
items = set()
with open('ml-100k/ua.base', 'rt') as f:
    for line in f:
        cols = line[:-1].split()
        user = int(cols[0]) - 1
        item = int(cols[1]) - 1
        # The rating is 1 to 5, but let's rescale to [0,1]
        rating = (int(cols[2])-1) / 4.0
        training_user_item_rating_tuples.append((user, item, rating))
        users.add(user)
        items.add(item)
        
test_user_item_rating_tuples = []
with open('ml-100k/ua.test', 'rt') as f:
    for line in f:
        cols = line[:-1].split()
        user = int(cols[0]) - 1
        item = int(cols[1]) - 1
        # The rating is 1 to 5, but let's rescale to [0,1]
        rating = (int(cols[2])-1) / 4.0
        test_user_item_rating_tuples.append((user, item, rating))      

In [5]:
training_user_item_rating_tuples[0]

(233, 482, 1.0)

In [6]:
[x for x  in training_user_item_rating_tuples if x[0]==233]

[(233, 482, 1.0),
 (233, 126, 0.75),
 (233, 225, 0.25),
 (233, 602, 0.75),
 (233, 523, 0.5),
 (233, 44, 0.75),
 (233, 284, 0.75),
 (233, 178, 0.5),
 (233, 132, 0.5),
 (233, 191, 0.5),
 (233, 147, 0.5),
 (233, 299, 0.5),
 (233, 866, 0.75),
 (233, 1447, 0.5),
 (233, 366, 0.75),
 (233, 47, 0.25),
 (233, 1453, 0.5),
 (233, 645, 0.5),
 (233, 184, 0.5),
 (233, 418, 0.75),
 (233, 627, 0.25),
 (233, 95, 0.25),
 (233, 1368, 0.5),
 (233, 1169, 0.0),
 (233, 81, 0.5),
 (233, 969, 0.75),
 (233, 218, 0.25),
 (233, 13, 0.5),
 (233, 241, 0.75),
 (233, 300, 0.5),
 (233, 1197, 0.5),
 (233, 600, 0.5),
 (233, 11, 0.0),
 (233, 505, 0.75),
 (233, 988, 0.25),
 (233, 1199, 0.5),
 (233, 233, 0.75),
 (233, 287, 0.5),
 (233, 171, 0.5),
 (233, 646, 0.5),
 (233, 415, 0.75),
 (233, 217, 0.25),
 (233, 693, 0.5),
 (233, 1284, 0.5),
 (233, 1459, 0.5),
 (233, 655, 0.75),
 (233, 1446, 0.5),
 (233, 1119, 0.5),
 (233, 426, 0.75),
 (233, 515, 0.5),
 (233, 873, 0.0),
 (233, 312, 0.75),
 (233, 380, 0.5),
 (233, 232, 0.25),
 

In [15]:
# We'll create a factorization model with 20-dimensinal latent factors for users and items
model = MatrixFactorization(943, 1682, n_factors=3)

# Use Mean Squared Error (MSE) loss to decide how wrong our predictions are
loss_fn = torch.nn.MSELoss()

# This is stochastic gradient descent
optimizer = torch.optim.SGD(model.parameters(),
                            lr=1e-6)

num_epochs=1

# Tell pytorch we're going to train the model
model.train()

for epoch in range(num_epochs):
    random.shuffle(training_user_item_rating_tuples)

    for user, item, rating in tqdm(training_user_item_rating_tuples):
        optimizer.zero_grad()    

        # Get user, item and rating data and put them in a pytorch Tensor object
        rating = Variable(torch.FloatTensor([rating]))
        user = Variable(torch.LongTensor([user]))
        item = Variable(torch.LongTensor([item]))

        # Predict the rating. Note that this is *implicitly* calling .forward(user, item)
        # The notation seems weird at first, but this was adopt to remind everyone
        # that nerural network are themselves _functions_ over their inputs!
        prediction = model(user, item)

        # The loss function (defined above) figures out how wrong the prediction was
        loss = loss_fn(prediction, rating)

        # Backpropagate the error in the loss through the network 
        # to figure out what needs to change
        loss.backward()

        # Update the weights in the network using our particular optimizer
        optimizer.step()

In [16]:
model.user_factors(torch.LongTensor([233]))

tensor([[ 0.5623, -0.3431, -0.1746]], grad_fn=<EmbeddingBackward>)

In [17]:
model.item_factors(torch.LongTensor([482]))

tensor([[ 0.2111, -1.3357, -0.9328]], grad_fn=<EmbeddingBackward>)

In [18]:
(model.user_factors(torch.LongTensor([233])) 
 * model.item_factors(torch.LongTensor([482]))).sum(1)

tensor([0.7398], grad_fn=<SumBackward1>)

In [ ]:
# Tell pytorch we're going to evaluate, so don't try to learn
model.eval()

pred_ratings = []
gold_ratings = []
for user, item, rating in tqdm(test_user_item_rating_tuples):

    # Get user and item and put them in a pytorch Tensor object
    user = Variable(torch.LongTensor([user]))
    item = Variable(torch.LongTensor([item]))

    # Predict the score again
    prediction = model(user, item)
    
    # Get the value as a python float object
    prediction = prediction.detach().numpy()[0]
    
    pred_ratings.append(prediction)
    gold_ratings.append(rating)

In [ ]:
print(mean_squared_error(gold_ratings, pred_ratings, squared=False))